In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import json
import uuid

es = Elasticsearch(["http://localhost:9200"])

In [ ]:
import os
import sys

sys.path.insert(0, os.path.join(os.pardir, 'ml_service'))

In [ ]:
artifacts_dir = os.path.join(os.getcwd(), os.pardir, os.pardir, 'additional_data')

In [ ]:
os.environ['EMB_PATH_GLOVE'] = os.path.join(artifacts_dir, 'glove.6B.50d.txt')
os.environ['EMB_PATH_KNRM'] = os.path.join(artifacts_dir, 'embeddings.bin')
os.environ['MLP_PATH'] = os.path.join(artifacts_dir, 'knrm_mlp.bin')
os.environ['VOCAB_PATH'] = os.path.join(artifacts_dir, 'vocab.json')

In [ ]:
from main import MLService
from utils import RankingDataset, CandidateModel, collate_fn

In [ ]:
import utils
from importlib import reload
reload(utils)

In [ ]:
candidate_model = CandidateModel(es, 'question_base')

In [ ]:
q = 'How to learn to skate?'

In [ ]:
ml_service = MLService()

In [ ]:
def pipeline(query):
    candidates = candidate_model.query(query, size=20)
    candidates = [i[1] for i in candidates]
    ds = RankingDataset(query=query,
                candidates=candidates,
                vocab=ml_service.vocab,
                oov_val=ml_service.vocab['OOV'],
                preproc_func=ml_service.simple_preproc)
    dl = torch.utils.data.DataLoader(
        ds, 
        batch_size=ml_service.dataloader_bs, 
        num_workers=0,
        collate_fn=collate_fn, 
        shuffle=False)
    
    score = ml_service.predict(dl)
    return np.array(candidates)[np.argsort(score)]

In [ ]:
pipeline('How to learm python?')

In [ ]:
import redis

redis = redis.from_url("redis://localhost:6379")

In [ ]:
test_result = {'result': pipeline('How to learm python?').tolist()}

In [ ]:
result_data = json.dumps(test_result)

In [ ]:
redis.set(request_id, result_data, ex=300)